# Spot the oops

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, average_precision_score, roc_curve, brier_score_loss

np.random.seed(None)
plt.rcParams['figure.figsize'] = (6,4)


In [ ]:
# Generating data.
N_sessions = 12; flips_per_session = 80
p_A, p_B = 0.5, 0.7
rows = []
for s in range(N_sessions):
    coin = 'A' if s % 2 == 0 else 'B'
    p = p_A if coin=='A' else p_B
    drift = np.random.normal(0, 0.03)
    for t in range(flips_per_session):
        spin = np.random.normal(0,1)
        y = np.random.rand() < (p + drift)
        rows.append((s, t, coin, spin, int(y)))
df = pd.DataFrame(rows, columns=['session','t','coin','spin','y'])
df.head()


In [ ]:
X = df[['t']].values; y = df['y'].values
poly = Pipeline([('poly', PolynomialFeatures(degree=10, include_bias=False)),
                      ('sc', StandardScaler(with_mean=False)),
                      ('lr', LogisticRegression(max_iter=2000))])
poly.fit(X, y)
proba = poly.predict_proba(X)[:,1]
plt.scatter(df['t'], proba, s=6, alpha=0.5)
plt.xlabel('t'); plt.ylabel('P(head)'); plt.show()
print("Training acc:", accuracy_score(y, (proba>=0.5).astype(int)))

In [ ]:
train_df = df[df['coin']=='B']; test_df = df[df['coin']=='A']
Xtr = train_df[['spin']].values; ytr = train_df['y'].values
Xte = test_df[['spin']].values; yte = test_df['y'].values
pipe = Pipeline([('sc', StandardScaler(with_mean=False)),
                 ('lr', LogisticRegression(max_iter=1000))]).fit(Xtr, ytr)
pred_te = pipe.predict_proba(Xte)[:,1]
print("Test acc:", accuracy_score(yte, (pred_te>=0.5).astype(int)))


In [ ]:
X = df[['spin']].values; y = df['y'].values
kf = KFold(n_splits=5, shuffle=True)
scores = []
for tr, te in kf.split(X):
    clf = Pipeline([('sc', StandardScaler(with_mean=False)),
                    ('lr', LogisticRegression(max_iter=500))]).fit(X[tr], y[tr])
    proba = clf.predict_proba(X[te])[:,1]
    scores.append(roc_auc_score(y[te], proba))
print("CV AUC:", np.mean(scores))


In [ ]:
from scipy.stats import binom_test
def simulate_until_significant(max_iter=200):
    for _ in range(max_iter):
        samp = df[df['coin']=='A'].sample(40, replace=False)
        k = int(samp['y'].sum()); n = len(samp)
        pval = binom_test(k, n, 0.5)  # PITFALL 10: we only report when p<0.05
        if pval < 0.05: return k, n, pval
    return None
print("Result:", simulate_until_significant())


In [ ]:
n = 1500
y_imb = (np.random.rand(n) < 0.05).astype(int)
pred_always_tail = np.zeros_like(y_imb)
acc = accuracy_score(y_imb, pred_always_tail)
print("Accuracy", acc)

proba_trivial = np.full_like(y_imb, 0.01, dtype=float)
prec, rec, _ = precision_recall_curve(y_imb, proba_trivial)
ap = average_precision_score(y_imb, proba_trivial)
plt.plot(rec, prec, label=f"Trivial (AP={ap:.3f})")
plt.axhline(y_imb.mean(), ls='--', label=f"Baseline precision={y_imb.mean():.2f}")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.legend(); plt.show()


In [ ]:
scaler = StandardScaler().fit(df[['spin']])
X_all = scaler.transform(df[['spin']])
y_all = df['y'].values

Xtr_l, Xte_l, ytr_l, yte_l = train_test_split(X_all, y_all, test_size=0.3, random_state=1)
bad_clf = LogisticRegression(max_iter=1000).fit(Xtr_l, ytr_l)
pred_bad = bad_clf.predict_proba(Xte_l)[:,1]
print("AUC:", roc_auc_score(yte_l, pred_bad))


In [ ]:
X = pd.get_dummies(train_df[['coin']], drop_first=True).values
y = train_df['y'].values
Xv = pd.get_dummies(test_df[['coin']], drop_first=True).values
yv = test_df['y'].values

Cs = np.logspace(-3, 3, 40)
val_scores = []
for C in Cs:
    m = LogisticRegression(max_iter=1000, C=C).fit(X, y)
    val_scores.append(roc_auc_score(yv, m.predict_proba(Xv)[:,1]))
C_star = Cs[np.argmax(val_scores)]


In [ ]:
Xa = pd.get_dummies(df[['coin']], drop_first=True).values
ya = df['y'].values
lin = LinearRegression().fit(Xa, ya)
yhat = lin.predict(Xa)
mse = ((ya - yhat)**2).mean()
print("Linear regression MSE", mse)
plt.hist(yhat, bins=30); plt.xlabel("Predicted value"); plt.show()


In [ ]:

def run_once(seed):
    tr, te = train_test_split(df, test_size=0.3, random_state=seed, stratify=df['coin'])
    Xtr = pd.get_dummies(tr[['coin']], drop_first=True).values
    ytr = tr['y'].values
    Xte = pd.get_dummies(te[['coin']], drop_first=True).values
    yte = te['y'].values
    model = LogisticRegression(max_iter=1000).fit(Xtr, ytr)
    return roc_auc_score(yte, model.predict_proba(Xte)[:,1])

seeds = range(50)
scores = [run_once(s) for s in seeds]
print("Result:", np.max(scores))
#plt.plot(seeds, scores, marker='o'); plt.xlabel("Seed"); plt.ylabel("AUC"); plt.show()


In [ ]:
rates = df.groupby('coin')['y'].mean()
plt.bar(rates.index, rates.values)
plt.ylim(0.55, 0.75)
plt.ylabel("Heads rate"); plt.show()
